# Synchronisation BDLex-Verbes3
- Division des fréquences pour les homophones

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [52]:
# -*- coding: utf8 -*-
import codecs
import re
import csv
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pandas as pd
debug=0

# Préparation du lexique

### Lecture des lexiques

In [53]:
nomLexique="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/bdlexique-PDM.txt"
with codecs.open(nomLexique, 'r',encoding='utf8') as infile:
    bdlexique=infile.readlines()
for ligne in bdlexique[:5]:
    (ortho,phono,ext,cs,ms,vs,lexeme,L23,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms)=ligne.strip().split(";")
    print ortho,cs,ms,vs,lexeme,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms

a N Mj  a 8136 5865 8136 5865
à p   à 1219040 1920905 1219040 1920905
abaca N MS  abaca 1 0 1 0
abacas N MP  abaca *** *** *** ***
abacule N MS  abacule *** *** *** ***


In [67]:
nomVerbes3FreqCases="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/Verbes3-FrequencesCases.csv"
verbes3FreqCases=pd.read_csv(nomVerbes3FreqCases,encoding="utf8",sep="\t")
verbes3FreqCases.columns=["case","films","textes"]
listCases=verbes3FreqCases["case"].tolist()

In [68]:
def ratioCases(chaine):
    ratioFilms={}
    ratioTextes={}
    if chaine=="*":
        cases=listCases
    else:
        cases=chaine.strip(";").split(";")        
    totalCasesFilms=verbes3FreqCases[verbes3FreqCases["case"].isin(cases)]["films"].sum()
    totalCasesTextes=verbes3FreqCases[verbes3FreqCases["case"].isin(cases)]["textes"].sum()
    if debug: print totalCasesFilms,totalCasesTextes
    for case in cases:
        ratioFilms[case]=verbes3FreqCases[verbes3FreqCases["case"]==case]["films"].values[0]/totalCasesFilms
        ratioTextes[case]=verbes3FreqCases[verbes3FreqCases["case"]==case]["textes"].values[0]/totalCasesTextes
    return ratioFilms,ratioTextes

In [84]:
globRatioFilms,globRatioTextes=ratioCases("*")
globRatioFilms,globRatioTextes

({u'cnd:pre:1p': 0.00045687616266769891,
  u'cnd:pre:1s': 0.0075901479292838263,
  u'cnd:pre:2p': 0.0015875236918820805,
  u'cnd:pre:2s': 0.0007041404029089663,
  u'cnd:pre:3p': 0.00082035078854995587,
  u'cnd:pre:3s': 0.009118621161451906,
  u'imp:pre:1p': 0.0010517409537562684,
  u'imp:pre:2p': 0.0078922167155084412,
  u'imp:pre:2s': 0.0051775283741068686,
  u'ind:fut:1p': 0.0016776992716495687,
  u'ind:fut:1s': 0.0084709598845192861,
  u'ind:fut:2p': 0.0027622256966228927,
  u'ind:fut:2s': 0.0039948221925985312,
  u'ind:fut:3p': 0.0022792765522575589,
  u'ind:fut:3s': 0.013998855889393619,
  u'ind:imp:1p': 0.0010358001395882274,
  u'ind:imp:1s': 0.017771560933389337,
  u'ind:imp:2p': 0.0028540489832137943,
  u'ind:imp:2s': 0.00049598602862616266,
  u'ind:imp:3p': 0.0026802701749891666,
  u'ind:imp:3s': 0.029425246770339846,
  u'ind:pas:1p': 3.7501750333712269e-05,
  u'ind:pas:1s': 0.00057224234141173356,
  u'ind:pas:2p': 5.5698164138107607e-05,
  u'ind:pas:2s': 0.0001453322162269349

In [103]:
def bdlex2verbes3(case):
    modes={"i":"ind","s":"sub","I":"imp","c":"cnd"}
    temps={"p":"pre","a":"pas","i":"imp","f":"fut"}
    if case[2:] in [p+n for p in "123" for n in "SP"]:
        return modes[case[1]]+":"+temps[case[0]]+":"+case[2:].lower()
    elif case=="inf":
        return "inf"
    elif case=="pP":
        return "par:pre"
    elif case[2:] in [g+n for g in "MF" for n in "SP"]:
        return "par:pas:"+case[2:].lower()

In [104]:
bdlex2verbes3("ppMS")

'par:pas:ms'

In [57]:
with codecs.open(nomVerbes3FreqCases, 'r',encoding='utf8') as infile:
    csvVerbes3FreqCases=infile.readlines()

for ligne in csvVerbes3FreqCases[1:]:
    case,filmsCF,textesCF=ligne.strip().split("\t")
    

In [79]:
nomVerbes3="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/Verbes3.csv"
verbes={}
vlemmes={}

with codecs.open(nomVerbes3, 'r',encoding='utf8') as infile:
    csvVerbes3=infile.readlines()

for ligne in csvVerbes3[1:]:
    morceaux=ligne.strip().split("\t")
    (ortho,phon,lemme,cgram,genre,nombre,freqlemfilms2,freqlemlivres,freqfilms2,freqlivres,infover)=morceaux[:11]
    freqlemfilms2=100*float(freqlemfilms2)
    freqlemlivres=100*float(freqlemlivres)
    freqfilms2=100*float(freqfilms2)
    freqlivres=100*float(freqlivres)
    if cgram=="VER":
#        print ortho,lemme,genre,nombre,infover
        if infover=="ind:pre:2p;inf;":
            infover="inf;"
        fRatioFilms,fRatioTextes=ratioCases(infover)
        tams=infover.rstrip(";").split(";")
        for tam in tams:
            morceaux=tam.split(":")
#            print morceaux
            if len(morceaux)==3:
                if morceaux[0]=="imp":
                    mode="I"
                else:
                    mode=morceaux[0][0]
                if morceaux[0]=="ind" and morceaux[1]=="pas":
                    temps="a"
                elif morceaux[1]=="pas":
                    temps="p"
                else:
                    temps=morceaux[1][0]
                infotam=temps+mode+morceaux[2].upper()
            elif len(morceaux)==2:
                if morceaux[1]=="pre":
                    infotam=morceaux[0][0]+"P"
            elif len(morceaux)==1:
                infotam=morceaux[0]
            verbe=";".join([ortho,lemme,infotam])
 #           print verbe
            verbes[verbe]=(str(freqlemfilms2*globRatioFilms[tam]),str(freqlemlivres*globRatioTextes[tam]),str(freqfilms2*fRatioFilms[tam]),str(freqlivres*fRatioTextes[tam]))
            if not lemme in vlemmes:
                vlemmes[lemme]=(freqlemfilms2,freqlemlivres)
#            print verbe, verbes[verbe]

In [80]:
verbes

{u'd\xe9barrasserez;d\xe9barrasser;fi2P': ('17.062268128',
  '6.63598016289',
  '16.0',
  '7.0'),
 u'extasiais;extasier;ii1S': ('1.20846614347',
  '16.4184046802',
  '0.0',
  '33.7910080787'),
 u'ressenties;ressentir;ppFP': ('17.6134968696',
  '62.253848507',
  '7.0',
  '34.0'),
 u'hypnotisais;hypnotiser;ii1S': ('6.62879222815',
  '9.42371446715',
  '0.0',
  '7.0'),
 u'd\xe9clam\xe2t;d\xe9clamer;is3S': ('0.00433462771498',
  '1.58476286455',
  '0.0',
  '7.0'),
 u'agace;agacer;pI2S': ('3.28773051756',
  '6.65416183907',
  '3.0013886455',
  '3.40803682724'),
 u'exerce;exercer;pi1S': ('166.954279271',
  '205.601676751',
  '110.628548767',
  '144.474411832'),
 u'comment\xe9es;commenter;ppFP': ('0.603661355012',
  '17.6891816473',
  '1.0',
  '47.0'),
 u'd\xe9nich\xe9es;d\xe9nicher;ppFP': ('1.13354187774',
  '7.79282563289',
  '1.0',
  '14.0'),
 u'craques;craquer;pi2S': ('135.985718661', '47.0691883777', '107.0', '14.0'),
 u'd\xe9capuchonne;d\xe9capuchonner;pi3S': ('0.0',
  '6.82579931061',


In [105]:
resync=[]
for ligne in bdlexique[:]:
    nouvelleLigne=ligne.strip()
    try :
        (ortho,phono,ext,cs,ms,vs,lexeme,L23,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms)=ligne.strip().split(";")
    except ValueError:
        print ligne
#    print ortho, lexeme, lemmeFrantext,lemmeFilms,formeFrantext,formeFilms
    if lexeme[-1] in "1234":
        lexeme=lexeme[:-1]
    if lexeme in [u"XXX"]: 
        debug=1
    else:
        debug=0
    if cs in ["V","K"]:
        if cs=="K": #Si la catégorie est K, il s'agit d'un adjectif/participe
            vs="pp"
            if not ms in ["MS", "MP", "FS", "FP"]:      #Si l'adjectif/participe est invariable (ij) mettre ms à MS
                ms="MS"
        verbe=";".join([ortho,lexeme,vs+ms])
        bdlCase=vs+ms
        case=bdlex2verbes3(bdlCase)
        if verbe in verbes:
            if lemmeFrantext!=verbes[verbe][0] and lemmeFilms!=verbes[verbe][1] and formeFrantext!=verbes[verbe][2] and formeFilms!=verbes[verbe][3]:
                if debug: print "different"
                if debug: print verbe, verbes[verbe]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,verbes[verbe][0],verbes[verbe][1],verbes[verbe][2],verbes[verbe][3],str(globRatioFilms[case]),str(globRatioTextes[case])])
        elif lexeme in vlemmes:
            if debug: print vlemmes[lexeme]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,str(vlemmes[lexeme][0]*globRatioFilms[case]),str(vlemmes[lexeme][1]*globRatioTextes[case]),"0","0",str(globRatioFilms[case]),str(globRatioTextes[case])])
        else:
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,"0","0","0","0",str(globRatioFilms[case]),str(globRatioTextes[case])])
    if debug: print [ortho, nouvelleLigne]
    resync.append(nouvelleLigne)
    

In [106]:
resync

[u'a;a;;N;Mj;;a;L23;8136;5865;8136;5865',
 u'\xe0;a;;p;;;\xe0;L23;1219040;1920905;1219040;1920905',
 u'abaca;abaka;;N;MS;;abaca;;1;0;1;0',
 u'abacas;abaka;z";N;MP;;abaca;;***;***;***;***',
 u'abacule;abakyl;@;N;MS;;abacule;;***;***;***;***',
 u'abacules;abakyl;@z";N;MP;;abacule;;***;***;***;***',
 u'abaissant;ab\xe8s\xe2;t";J;MS;;abaissant;;4;27;3;27',
 u'abaissants;ab\xe8s\xe2;z";J;MP;;abaissant;;***;***;***;***',
 u'abaissante;ab\xe8s\xe2t;@;J;FS;;abaissant;;4;27;1;0',
 u'abaissantes;ab\xe8s\xe2t;@z";J;FP;;abaissant;;***;***;***;***',
 u'abaisse;abEs;@;N;FS;;abaisse;;2;0;2;0',
 u'abaisses;abEs;@z";N;FP;;abaisse;;***;***;***;***',
 u'abaisse-langue;ab\xe8s6l\xe2g;@;N;Mj;;abaisse-langue;L23;4;14;4;14',
 u'abaissement;ab\xe8s6m\xe2;;N;MS;;abaissement;;31;216;31;216',
 u'abaissements;ab\xe8s6m\xe2;z";N;MP;;abaissement;;***;***;***;***',
 u'abaisser;ab\xe8se;R";V;;inf;abaisser;L23;84.7655296738;328.807523695;109.0;291.0;0.171938194064;0.182265811361',
 u'abaisse;abEs;@;V;1S;pi;abaisser;L2

In [107]:
nomLexiqueResync=nomLexique.split(".")[0]+"-Resync3.txt"
resync=sorted([x.rstrip("\n")+"\n" for x in set(resync)])
with codecs.open(nomLexiqueResync, 'w',encoding='utf8') as outfile:
    outfile.writelines(resync)

In [108]:
verbes[u"été;être;ppMS"]

('338091.482911', '181911.391051', '104122.0', '81899.0')

In [109]:
resync[436300:436400]

[u'\xe9tuv\xe9es;\xe8tyve;z";K;FP;pp;\xe9tuver;L23;0.00223578279634;0.301772291023;0;0;0.00223578279634;0.00887565561833\n',
 u'\xe9tuv\xe9es;\xe8tyve;z";N;FP;;\xe9tuv\xe9e;;***;***;***;***\n',
 u'\xe9tuv\xe9s;\xe8tyve;z";K;MP;pp;\xe9tuver;L23;0.0079411646715;0.628238865392;0;0;0.0079411646715;0.018477613688\n',
 u'\xe9tymologie;\xe8tim\xf2l\xf2Zi;;N;FS;;\xe9tymologie;L23;9;108;9;95\n',
 u'\xe9tymologies;\xe8tim\xf2l\xf2Zi;z";N;FP;;\xe9tymologie;L23;9;108;0;14\n',
 u'\xe9tymologique;\xe8tim\xf2l\xf2Zik;@;J;FS;;\xe9tymologique;;0;54;0;47\n',
 u'\xe9tymologique;\xe8tim\xf2l\xf2Zik;@;J;MS;;\xe9tymologique;;0;54;0;47\n',
 u'\xe9tymologiquement;\xe8tim\xf2l\xf2Zik6m\xe2;t";A;;Ma;\xe9tymologiquement;;1;14;1;14\n',
 u'\xe9tymologiques;\xe8tim\xf2l\xf2Zik;@z";J;FP;;\xe9tymologique;;0;54;0;7\n',
 u'\xe9tymologiques;\xe8tim\xf2l\xf2Zik;@z";J;MP;;\xe9tymologique;;0;54;0;7\n',
 u'\xe9tymologiste;\xe8tim\xf2l\xf2Zist;@;N;FS;;\xe9tymologiste;;0;14;0;14\n',
 u'\xe9tymologiste;\xe8tim\xf2l\xf2Zist;@;N